#### Search Tool

In [22]:
from langchain_community.tools.tavily_search import TavilySearchResults
TAVILY_API_KEY = "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"
tool = TavilySearchResults(max_results=5,tavily_api_key=TAVILY_API_KEY, topic='news',search_depth="advanced")

In [36]:
response = tool.invoke('notícias 2025 embraer')

In [37]:
response

[{'title': 'notícias - Embraer',
  'url': 'https://embraer.com/br/pt/noticias',
  'content': 'Tóquio, 4 de abril de 2025 – A Embraer (NYSE: ERJ/B3: EMBR3) promove no próximo dia 8, em Tóquio, o Embraer Aircraft Finance Summit, para que financiadores',
  'score': 0.8805613},
 {'title': 'Embraer delivered 30 new aircraft in the first quarter of 2025',
  'url': 'https://www.aerotime.aero/articles/embraer-delivers-20-more-aircraft-in-first-quarter-of-2025-versus-1q24',
  'content': 'On April 2, 2025, Embraer announced that it was continuing its active pursuit of industrial cooperation plans outside of Brazil.\xa0On April 1, 2025, the company signed a Memorandum of Understanding (MoU) with Turkish Aerospace (TUSAŞ-Türk Havacılık ve Uzay Sanayii), Turkey’s state-owned aerospace company, to cooperate in the industrial and research domains.\xa0The initiative could potentially involve the building of an assembly line for Embraer’s E2 family of commercial airliners in Turkey. [...] Embraer has r

In [42]:
from pprint import pprint
pprint(response[1]['content'])

('On April 2, 2025, Embraer announced that it was continuing its active '
 'pursuit of industrial cooperation plans outside of Brazil.\xa0On April 1, '
 '2025, the company signed a Memorandum of Understanding (MoU) with Turkish '
 'Aerospace (TUSAŞ-Türk Havacılık ve Uzay Sanayii), Turkey’s state-owned '
 'aerospace company, to cooperate in the industrial and research domains.\xa0'
 'The initiative could potentially involve the building of an assembly line '
 'for Embraer’s E2 family of commercial airliners in Turkey. [...] Embraer has '
 'released the company’s latest delivery figure for the first quarter of 2025 '
 '(January to March/1Q25) which indicates that the Brazilian planemaker is '
 'continuing to grow and increase its global reach through its range of civil '
 'and military aircraft. The announcement, made on April 2, 2025, shows that '
 'the manufacturer delivered 30 new aircraft (of all types and variants) '
 'during the first three months of 2025, a figure that represents 

In [ ]:
from langchain_community.document_loaders import WebBaseLoader

loader = WebBaseLoader(response[1]['url'])
text = loader.load()

[Document(metadata={'source': 'https://www.aerotime.aero/articles/embraer-delivers-20-more-aircraft-in-first-quarter-of-2025-versus-1q24', 'title': 'Embraer delivered 30 new aircraft in the first quarter of 2025', 'description': 'Embraer has announced it delivered 30 new aircraft to customers in the first quarter of 2025, a 20% over the same period in 2024.', 'language': 'en-US'}, page_content='\n\n\n\n\n\n\n\nEmbraer delivered 30 new aircraft in the first quarter of 2025\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n \n\n\n\n\n\n\nCivil Aviation\n\nAirlines\nAircraft\nAirport\nCargo\nMRO\n\n\nDefense\nExecutive Spotlight\nBusiness Aviation\nNewsletter signup\nSustainability\nTechnology\nSpace\nAviation Events\nAviation Safety\nAviation economics\nAviation Top 10s\nJobs\nAviation History\nProfile\nVideo\nAbout us\nAdvertise with us\nJoin our team\nOur Team\n\n\n\n\n\n\

In [56]:
text[0].metadata

{'source': 'https://www.aerotime.aero/articles/embraer-delivers-20-more-aircraft-in-first-quarter-of-2025-versus-1q24',
 'title': 'Embraer delivered 30 new aircraft in the first quarter of 2025',
 'description': 'Embraer has announced it delivered 30 new aircraft to customers in the first quarter of 2025, a 20% over the same period in 2024.',
 'language': 'en-US'}

In [2]:
from langchain_openai import ChatOpenAI
import dotenv
import os

dotenv.load_dotenv()

OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')


llm = ChatOpenAI(model_name="gpt-4o", temperature=0.3,openai_api_key=OPENAI_API_KEY)

In [3]:
model_with_tool = llm.bind_tools([tool])

In [4]:
from pydantic import BaseModel, Field
from typing import Literal, List
from langchain_core.tools import tool
from langgraph.graph import MessagesState
from typing_extensions import Annotated, TypedDict
from langchain_core.output_parsers import PydanticToolsParser

In [ ]:
from langchain.prompts import PromptTemplate
from langchain_core.prompts import ChatPromptTemplate

template =f"""
Você é um assistente de IA conversacional avançado, especializado em pesquisa na web em tempo real e recuperação de informações.
Seu objetivo é encontrar informações precisas, confiáveis e atualizadas a partir de fontes confiáveis e apresentá-las de forma clara e útil para o usuário.
Planeje as suas ações e pense passo a passo.

Você opera em um ciclo de Pensamento, Ação, PAUSA, Observação.
Ao final do ciclo, você fornece uma Resposta concisa e bem fundamentada ao usuário.

Use Pensamento para explicar seu raciocínio e o que pretende fazer em seguida.

Use Ação para executar uma das ferramentas disponíveis.

Após uma Ação, PAUSE e aguarde pela Observação.

Observação será o resultado retornado da sua ação.

Após reunir informações suficientes por meio desse ciclo, sintetize as descobertas em uma Resposta final e apresente-as com clareza.

"""

prompt_agent_test = ChatPromptTemplate.from_messages([
    ("system", template),
    ("user", "{input}"),
])

In [ ]:
teste = prompt_agent_test | model_with_tool

In [14]:
teste.invoke({'input':'cotação do dolar hoje'})

AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_DvyWkkmzb8TBzpdKOAhKtXvX', 'function': {'arguments': '{"query":"cotação do dólar hoje"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 23, 'prompt_tokens': 269, 'total_tokens': 292, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-2024-08-06', 'system_fingerprint': 'fp_898ac29719', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-9bb20160-bb5f-4512-b08c-e4ee73a0a9f6-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'cotação do dólar hoje'}, 'id': 'call_DvyWkkmzb8TBzpdKOAhKtXvX', 'type': 'tool_call'}], usage_metadata={'input_tokens': 269, 'output_tokens': 23, 'total_tokens': 292, 'input_token_details': {'audio': 0, 'cache_r

In [8]:
model_with_tool.invoke("cotação do dolar hoje")

AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_W8PdAxmWUTHVCxewhC2tgF5o', 'function': {'arguments': '{"query":"cotação do dólar hoje"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 23, 'prompt_tokens': 84, 'total_tokens': 107, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-2024-08-06', 'system_fingerprint': 'fp_92f14e8683', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-4ba98520-b0d7-44f5-b807-d958253db0fb-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'cotação do dólar hoje'}, 'id': 'call_W8PdAxmWUTHVCxewhC2tgF5o', 'type': 'tool_call'}], usage_metadata={'input_tokens': 84, 'output_tokens': 23, 'total_tokens': 107, 'input_token_details': {'audio': 0, 'cache_rea

In [15]:
from langgraph.graph import MessagesState
# Inherit 'messages' key from MessagesState, which is a list of chat messages
class AgentState(MessagesState):
    # Final structured response from the agent
    pass

In [16]:
from typing import Annotated

from langchain_anthropic import ChatAnthropic
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_core.messages import BaseMessage
from typing_extensions import TypedDict

from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages
from langgraph.prebuilt import ToolNode, tools_condition


class State(TypedDict):
    messages: Annotated[list, add_messages]


In [19]:
prompt_agent = ChatPromptTemplate.from_messages([
    ("system", template),
    ("placeholder", "{messages}"),
])

In [47]:
from langgraph.graph import StateGraph, END
from langgraph.prebuilt import ToolNode
from langchain_core.messages import HumanMessage
from langgraph.checkpoint.memory import MemorySaver
memory = MemorySaver()

# Define the function that calls the model
def call_model(state: State):
    chain = prompt_agent | model_with_tool
    response = chain.invoke(input={"messages":state['messages']})
    # We return a list, because this will get added to the existing list
    return {"messages": [response]}


# Define the function that determines whether to continue or not
def should_continue(state: State):
    messages = state["messages"]
    last_message = messages[-1]
    # If there is no function call, then we respond to the user
    if not last_message.tool_calls:
        return END
    # Otherwise if there is, we continue
    else:
        return "continue"


# Define a new graph
workflow = StateGraph(State)

# Define the two nodes we will cycle between
workflow.add_node("agent", call_model)
workflow.add_node("tools", ToolNode([tool]))

# Set the entrypoint as `agent`
# This means that this node is the first one called
workflow.set_entry_point("agent")

# We now add a conditional edge
workflow.add_conditional_edges(
    "agent",
    should_continue,
    {
        "continue": "tools",
        END: END,
    },
)

workflow.add_edge("tools", "agent")
graph = workflow.compile(checkpointer=memory)

In [ ]:
from IPython.display import Image, display
from langchain_core.runnables.graph import CurveStyle, MermaidDrawMethod, NodeStyles

display(
    Image(
        graph.get_graph().draw_mermaid_png(
            draw_method=MermaidDrawMethod.API,
        )
    )
)

In [29]:
from IPython.display import Image, display

try:
    display(Image(graph.get_graph().draw_mermaid_png()))
except Exception:
    # This requires some extra dependencies and is optional
    pass

In [48]:
config = {"configurable": {"thread_id": "de8f26l4"}}
inputs={"messages":HumanMessage("Qual a cotação do dolar hoje?")}

In [32]:
answer = graph.invoke(input=inputs,config=config)

In [49]:
events = graph.stream(
    input=inputs,
    config=config,
    stream_mode="values",
)
for event in events:
    event["messages"][-1].pretty_print()

================================ Human Message =================================

Qual a cotação do dolar hoje?
================================== Ai Message ==================================
Tool Calls:
  tavily_search_results_json (call_IiHhc1xtJdFKwsw0ke3zrV2a)
 Call ID: call_IiHhc1xtJdFKwsw0ke3zrV2a
  Args:
    query: cotação do dólar hoje
================================= Tool Message =================================
Name: tavily_search_results_json

[{"title": "Dólar Hoje: Cotação Comercial e Turismo. Valor e Preço!", "url": "https://dolarhoje.com/", "content": "Adicione ao Chrome\nGráfico do histórico do dólar\nValor do Dólar Hoje\nA cotação do dolar hoje está R$ 5,80 (+0,76% em relação a ontem). Cotação do dólar comercial americano atualizada sempre para que você fique informado sobre o dólar e tome as melhores decisões.\nO valor do dólar hoje é de caráter informativo.\nSobre o dólar\nO dólar dos Estados Unidos (US Dollar, USD, US$) é a moeda emitida pelos Estados Unidos atra